In [1]:
import pyscal.crystal_structures as pcs
import pyscal.csystem as pcm
import pyscal.core as pc
from ase import atoms
from ase.io import read, write
from ase.build import bulk
import numpy as np
import matplotlib.pyplot as plt

In [2]:
nx = 10
satoms, box = pcs.make_crystal(structure='bcc', 
    lattice_constant=3.127, 
    repetitions=(nx,nx,nx),)
print(len(satoms["positions"]))
sys = pc.System()
sys.box = box
sys.atoms = satoms


2000


In [3]:
sys.find_neighbors(method="cutoff", 
                   cutoff=3.6)

In [31]:
qlist = [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 
         20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,
        31, 32, 33, 34, 35, 36, 37, 38, 39, 40,
        41, 42, 43, 44, 45, 46, 47, 48, 49, 50]

In [32]:
%%timeit
sys.calculate_q(qlist, use_c=False)

18.3 s ± 114 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [33]:
%%timeit
sys.calculate_q(qlist, use_c=True)

13.1 s ± 231 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
